# Spline Bases
Much in the same way as a given number can be expressed as a string of digits in different bases,
a given spline can be expressed as a sequence of coefficients in different basis functions. The
generic form of a spline $f$ is

$$f:{\mathbb{R}}\rightarrow{\mathbb{R}},\;x\mapsto f(x)=
\sum_{k\in{\mathbb{R}}}\,c[k]\,\varphi(x-k).$$
There, the coefficients $c$ are to the spline $f$ what digits are to the string representation of
a number, and the basis $\varphi$ is to the spline what the decimal or binary base is to the
string representation of the number.

Any integer base greater than one will allow for the representation of nonnegative integer numbers.
Popular choices are ten (the familiar decimal representation) and two (the binary representation).
With functions, however, the basis $\varphi$ is not arbitrary but must satisfy technical
constraints—it must be a so-called Riesz basis. Moreover, one can have equivalent representations

$$f:{\mathbb{R}}\rightarrow{\mathbb{R}},\;x\mapsto f(x)=
\sum_{k\in{\mathbb{R}}}\,c_{1}[k]\,\varphi_{1}(x-k)=
\sum_{k\in{\mathbb{R}}}\,c_{2}[k]\,\varphi_{2}(x-k)$$
only if $\varphi_{1}$ and $\varphi_{2}$ are somehow related.

## Interpolating Cardinal B-Spline
The splines handled by this library are made of polynomial pieces of unit length and same degree,
which sets constraints on $\varphi.$ Despite these constraints, many equivalent bases exist, some
being more suitable than others. One very good basis is the B-spline $\varphi_{1}=\beta^{n},$ whose
superscript $n$ indicates its polynomial degree. It has many favorable properties but comes at a
price: it is not interpolating, which means that it is not trivial to determine the coefficients
that ensure that the spline takes imposed values. More precisely, given the sequence
$\left(y[q]\right)_{q\in{\mathbb{Z}}}$, some work is required to find $c_{1}$ such that

$$\left(y[q]\right)_{q\in{\mathbb{Z}}}=
\left(\sum_{k\in{\mathbb{R}}}\,c_{1}[k]\,\beta^{n}(q-k)\right)_{q\in{\mathbb{Z}}}.$$

Meanwhile, there exists an interpolating basis $\varphi_{2}=\eta^{n},$ called a cardinal B-spline.
Some of its mathematical properties can be considered less favorable than those of B-splines; most
notably, contrarily to B-splines, the support of cardinal B-splines is infinite, which makes it
an impractical basis. Yet, its interpolating property (according to which $\eta^{n}(0)=1$ and
$\eta^{n}(k)=0$ for $k\in{\mathbb{Z}}\setminus\{0\}$) greatly facilitates the determination
of the coefficients of the representation. Indeed, it is enough to let $c_{2}=y$ to automatically get that

$$\left(y[q]\right)_{q\in{\mathbb{Z}}}=
\left(\sum_{k\in{\mathbb{R}}}\,y[k]\,\eta^{n}(q-k)\right)_{q\in{\mathbb{Z}}}.$$

Here, we plot a cardinal B-spline, indexed by its degree. The hollow (blue or red) circles
indicate the samples at the integers. The function is made of polynomial pieces that lie between
so-called knots. The small filled (black) discs indicate the location of the knots of the spline.

In [ ]:
# Load the required libraries.
from ipywidgets import interactive
import matplotlib.pyplot as plt

import splinekit as sk # This library

# Define the plot function
def cardinal_b_spline_plot (
    degree = 3
):
    # A cardinal B-spline has an infinite support but decays exponentially
    # Here, we combine periodization and a wide margin to simplify the display effort
    supp = sk.interval.Closed((-200, 200))
    period = supp.diameter
    # Construct a spline from a periodized cardinal B-spline
    s = sk.PeriodicSpline1D.periodized_cardinal_b_spline(
        period = int(period),
        degree = degree
    )
    # Plot the cardinal B-spline
    s.plot(
        plt.subplots(),
        plotdomain = sk.interval.Closed((-8, 8)),
        plotrange = sk.interval.Closed((-0.25, 1.05)),
        plotpoints = 200 + 1
    )

# Interact with the degree
interactive(cardinal_b_spline_plot, degree = (0, 30))

## Relation to the Cardinal Sine Function
The function
$f:{\mathbb{R}}\rightarrow{\mathbb{R}},\;x\mapsto\sum_{k\in{\mathbb{R}}}\,c[k]\,\varphi(x-k)$
is a spline only for certain bases $\varphi.$ A famous case where $f$ fails to be a spline arises
if one sets $\varphi_{3}(x)=\sin(\pi\,x)/\left(\pi\,x\right)={\mathrm{sinc}}\,x,$ which is the
basis associated to the celebrated Whittaker–Shannon interpolation formula and has strong
theoretical properties.

As it turns out, the cardinal B-spline $\eta^{n}$ tends (in a functional sense) to ${\mathrm{sinc}}$ when $n\rightarrow\infty.$ The convergence, however, is poor. In particular, the
decay of the tails of the ${\mathrm{sinc}}$ function is reciprocal, which is much gentler than the
exponential decay of the tails of the cardinal B-spline.

We verify now visually over the range $x\in[-15,15]$ that
${\mathrm{sinc}}\,x=\lim_{n\rightarrow\infty}\eta^{n}(x).$ In the top plot, we show the
ground-truth ${\mathrm{sinc}}$ function in thin green, and the cardinal-spline approximation in
thicker blue. In the bottom plot, we show in thicker blue the difference between the
${\mathrm{sinc}}$ function and the cardinal-spline approximation.

In [ ]:
# Load the required libraries.
from ipywidgets import interactive
import math
import matplotlib.pyplot as plt
import numpy as np

import splinekit as sk # This library

# Define the plot function
def cardinal_b_spline_as_sinc_plot (
    degree = 3
):
    # Location of the samples
    abscissa = np.linspace(-15, 15, num = 200 + 1, dtype = float)
    # Sinc data
    def sinc (
        x
    ):
        if math.isclose(
            0,
            x,
            rel_tol = math.sqrt(math.ulp(1.0)),
            abs_tol = math.sqrt(math.ulp(1.0))
        ):
            return 1
        return math.sin(math.pi * x) / (math.pi * x)
    sinc_data = np.array([sinc(x) for x in abscissa], dtype = float)
    # Cardinal B-spline
    spline_data = np.array(
        [sk.cardinal_b_spline(x, degree) for x in abscissa],
        dtype = float
        
    )

    # Spline plot
    plt.subplot(211)
    plt.xlim(-15.025, 15.025)
    plt.ylim(-0.25, 1.05)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot(abscissa, sinc_data, "-C2", linewidth = 0.5)
    plt.plot(abscissa, spline_data, "-C0")

    # Difference plot
    plt.subplot(212)
    plt.xlim(-15.025, 15.025)
    plt.ylim(-0.2, 0.2)
    ax = plt.gca()
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    plt.plot([-15, 15], [0, 0], "-C2", linewidth = 0.5)
    plt.plot(abscissa, sinc_data - spline_data, "-C0")

    # Show the plot
    plt.show()

# Interact with the degree
interactive(cardinal_b_spline_as_sinc_plot, degree = (0, 30))